In [ ]:
import six
from google.cloud import translate_v2 as translate
import pandas as pd
import numpy as np

In [ ]:
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS']="KEY_PATH"

In [ ]:
def translate_text(target, text):
    """Translates text into the target language.

    Target must be an ISO 639-1 language code.
    See https://g.co/cloud/translate/v2/translate-reference#supported_languages
    """

    translate_client = translate.Client()

    if isinstance(text, six.binary_type):
        text = text.decode("utf-8")

    # will return a sequence of results for each text.
    result = translate_client.translate(text, target_language=target)


    return result["input"],result["translatedText"]

In [ ]:
input,output=translate_text('te', 'how are you doing')
print('Input Sentence in English :',input)
print('Output Sentence in Telugu :',output)

Input Sentence in English : how are you doing
Output Sentence in Telugu : నువ్వు ఎలా ఉన్నావు


In [ ]:
eng_hin=pd.read_csv('eng_hin.csv') # Using the english to hindi dataset for getting english sentences
eng_hin.dropna(inplace=True)
eng_hin.drop(['source','hindi_sentence'],axis=1,inplace=True)
eng_hin.head()

,english_sentence
0,politicians do not have permission to do what ...
1,"I'd like to tell you about one such child,"
2,This percentage is even greater than the perce...
3,what we really mean is that they're bad at not...
4,.The ending portion of these Vedas is called U...


In [ ]:
eng_tel=[]
length=0
for index,row in eng_hin.iterrows():
  length+=len(row['english_sentence'])
  if length>499000: # Google Translate Api charges for the translation above 500,000 characters, to avoid charges limiting translations to 499k characters.
    break
  input,output=translate_text('te', row['english_sentence'])
  eng_tel.append([input,output])

In [ ]:
eng_tel_df=pd.DataFrame(eng_tel,columns=['English','Telugu'])
eng_tel_df.head()

,English,Telugu
0,politicians do not have permission to do what ...,రాజకీయ నాయకులకు చేయవలసినది చేయడానికి అనుమతి లేదు.
1,"I'd like to tell you about one such child,",అలాంటి ఒక పిల్లల గురించి నేను మీకు చెప్పాలనుకు...
2,This percentage is even greater than the perce...,ఈ శాతం భారతదేశంలో ఉన్న శాతం కంటే ఎక్కువ.
3,what we really mean is that they're bad at not...,మేము నిజంగా అర్థం ఏమిటంటే వారు శ్రద్ధ చూపకపోవడ...
4,.The ending portion of these Vedas is called U...,.ఈ వేదాల ముగింపు భాగాన్ని ఉపనిషత్తు అంటారు.


In [ ]:
eng_tel_df.shape

(5615, 2)

In [ ]:
eng_tel_df.to_csv('eng_tel.csv',index=False)